In [3]:
%load_ext autoreload
%autoreload 2

Load model

In [34]:
from aiutils import load
import os

In [48]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [49]:
model_name = 'bm-bm-logistic-pred-1k-10'
model_path = os.path.join(model_dir, model_name)

In [36]:
import torch

In [107]:
model = torch.load(model_path, map_location='cpu')
model

PredictorMLP(
  (fc1): Linear(in_features=21, out_features=11, bias=True)
  (fc2): Linear(in_features=11, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): LeakyReLU(negative_slope=0.01)
  (tanh): Tanh()
)

In [37]:
from config import GameConfig 
from constants import BUY
from enums import StartingSplit
from player import load_players
from env import DefaultEnvironment
from simulations import simulate
import logging

In [38]:
from player import HeuristicPlayer, PredictorMLPPlayer
from buyagenda import BigMoneyBuyAgenda

In [52]:
logger = logging.getLogger()
# logger.setLevel(logging.ERROR)
logger.setLevel(BUY)

In [50]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['LOG', 'BM'], [model_path], map_location='cpu')
agent = players[0]
# players = [PredictorMLPPlayer(model=torch.load(model_path, # map_location='cpu'), train=False), HeuristicPlayer(BigMoneyBuyAgenda())]
env = DefaultEnvironment(config, players, logger=logger)

In [53]:
simulate(env, 100, None)

100%|██████████| 100/100 [00:01<00:00, 88.33it/s]===SUMMARY===
{0: 53, 1: 75, 'ProvinceWins': 100, 'Ties': 28}



Let's analyze the neural network outputs for some simple cases.

In [42]:
from state import ReducedStateFeature

In [43]:
copper = ReducedStateFeature.default_sandbox_feature()
silver = ReducedStateFeature.default_sandbox_feature()
estate = ReducedStateFeature.default_sandbox_feature()

In [44]:
copper[0] = 45
copper[7] = 8
copper

tensor([45., 10.,  8.,  8.,  8., 40., 30.,  8.,  0.,  3.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [45]:
silver[5] = 39
silver[7+5] = 1
silver

tensor([46., 10.,  8.,  8.,  8., 39., 30.,  7.,  0.,  3.,  0.,  0.,  1.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [46]:
estate[2] = 7
estate[7+2] = 4
estate

tensor([46., 10.,  7.,  8.,  8., 40., 30.,  7.,  0.,  4.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [54]:
agent.model(copper), agent.model(silver), agent.model(estate)

TypeError: 'LogisticRegression' object is not callable